# Artwork sequence prediction

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os

In [3]:
CONFIG_PATH = '/root/work/artwork_sequence/train_test_configuration/multivariate'
#CONFIG_PATH = '/root/work/artwork_sequence/train_test_configuration/univariate'

In [4]:
window_index = 3

museum_sequence_path = {
    'x_train' : os.path.join(CONFIG_PATH, 'X_train.csv'),
    'x_test' : os.path.join(CONFIG_PATH, 'X_test.csv'),
    'x_train_matrix' : os.path.join(CONFIG_PATH, 'X_train_matrix.npy'),
    'x_test_matrix' : os.path.join(CONFIG_PATH, 'X_test_matrix.npy'),
    'weights_folder' : os.path.join(CONFIG_PATH, 'config_'+str(window_index)+'/trained_model_weights')
}
museum_sequence_path

{'weights_folder': '/root/work/artwork_sequence/train_test_configuration/multivariate/config_3/trained_model_weights',
 'x_test': '/root/work/artwork_sequence/train_test_configuration/multivariate/X_test.csv',
 'x_test_matrix': '/root/work/artwork_sequence/train_test_configuration/multivariate/X_test_matrix.npy',
 'x_train': '/root/work/artwork_sequence/train_test_configuration/multivariate/X_train.csv',
 'x_train_matrix': '/root/work/artwork_sequence/train_test_configuration/multivariate/X_train_matrix.npy'}

## Load data

In [5]:
df_x_train = pd.read_csv(museum_sequence_path['x_train'], index_col=0)
df_x_test = pd.read_csv(museum_sequence_path['x_test'], index_col=0)
x_train_matrix = np.load(museum_sequence_path['x_train_matrix'])
x_test_matrix = np.load(museum_sequence_path['x_test_matrix'])
df_x_train.head()

,tour_path
20,/root/work/datasets/artwork_sequence/rijksmuse...
7,/root/work/datasets/artwork_sequence/rijksmuse...
40,/root/work/datasets/artwork_sequence/prado_cra...
0,/root/work/datasets/artwork_sequence/rijksmuse...
23,/root/work/datasets/artwork_sequence/prado_cra...


In [6]:
x_train_matrix.shape

(637, 300)

## Reset Tensorflow session

In [7]:
tf.keras.backend.clear_session()

## Config data to fit with the model input

Because the **Prediction feature model** split the data into training and validation dataset, it is necessary to give all the data in only one block

**Define timeline**

In [8]:
time = np.arange(x_train_matrix.shape[0] + x_test_matrix.shape[0])
time.shape

(859,)

**Define configuration to deal with the windowed dataset**

In [9]:
split_time = x_train_matrix.shape[0]

X = np.concatenate((x_train_matrix, x_test_matrix))

#length of the history
window_size = window_index

#Number of artwork's feature
n_features = X.shape[1]

#Number of feature to take into account
n_influence_features=3

batch_size = 128
shuffle_buffer_size = 300

## Create models

**Define model**

In [10]:
from Prediction_model_feature import Prediction_model_feature
from Prediction_model_feature_multivariate import Prediction_model_feature_multivariate

In [ ]:
from IPython.display import clear_output
import time

models = []
start_time = time.time()
for i in range(n_features):
    clear_output(wait=True)
    print("---------- Feature %s -------------" % (i))
    
    model_prediction = Prediction_model_feature_multivariate(
        X=X,
        split_time=split_time,
        train_batch_size=batch_size, 
        val_batch_size=batch_size, 
        window_size=window_size, 
        shuffle_buffer=shuffle_buffer_size,
        name="feature " + str(i),
        index = i,
        n_influence_features=n_influence_features,
        n_features=1)
    model_prediction.define_model(conv_filter=32, lstm_filter=64, dense_filter=16)
    model_prediction.train_model(epochs=40, lr=1e-6)
    models.append(model_prediction)
    
    #Save weights
    model_prediction.save_weights(museum_sequence_path)
    
print("--- %s seconds ---" % (time.time() - start_time))

---------- Feature 164 -------------


In [ ]:
len(models)

In [ ]:
models[0].model.summary()

## Evaluate model

### Load decoder model

In [ ]:
from tensorflow.python.keras.models import load_model

In [ ]:
decoder_model = load_model('wasserstein_decoder.h5')
decoder_model.summary()

**Test decoder with example**

In [ ]:
p = decoder_model.predict(X[1].reshape((1,1,1,-1)))

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(p[0][...,::-1])

### Predict features

**Sliced validation dataset**

In [ ]:
def validation_dataset(series, window_size, batch_size):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.map(lambda w: (w[:-1], w[-1:]))
    return ds.batch(batch_size).prefetch(1)

**Predict features**

In [ ]:
prediction = []

for i in range(n_features):
    val_dataset = validation_dataset(x_valid[:,i], window_size, batch_size)
    for x, y in val_dataset.take(1):
        prediction_feature = models[i].model.predict(x)[0]
        prediction.append(prediction_feature)

In [ ]:
code = []
for p in prediction:
    code.append(prediction[0][0])

code = np.array(code)

In [ ]:
code = code.reshape((-1,))

**Decode code**

In [ ]:
p = decoder_model.predict(code.reshape((1,1,1,-1)))

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(p[0])

In [ ]:
valid_decode = decoder_model.predict(x_valid[0].reshape((1,1,1,-1)))

In [ ]:
plt.imshow(valid_decode[0][...,::-1])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

cosine_distances(code, x_valid[0][...,::-1])

### Plot feature prediction

In [ ]:
from utils_plot import plot_series, plot_train_history, plot_prediction
import tensorflow as tf

In [ ]:
val_dataset = validation_dataset(x_valid[:,0], window_size, batch_size)
for x, y in val_dataset.take(1):
    prediction = models[0].model.predict(x)[0]
    plot = plot_prediction([x[0].numpy(), y[0].numpy(), prediction[0]] , 'Simple LSTM model')
    plot.show()

In [ ]:
val_dataset = validation_dataset(x_valid[:,20], window_size, batch_size)
for x, y in val_dataset.take(1):
    prediction = models[20].model.predict(x)[0]
    plot = plot_prediction([x[0].numpy(), y[0].numpy(), prediction[0]] , 'Simple LSTM model')
    plot.show()